In [1]:
from src.dataset import ConnTextULDataset
import pandas as pd
import torch as pt
import glob
import json
import tqdm
import csv
from time import time
%reload_ext autoreload
%autoreload 2

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [2]:
# Load in a dataset to access Matt's Traindata class which
# is embedded inside the phonology tokenizer (consider changing this)
ds = ConnTextULDataset(None, which_dataset="all")
print(ds.data_filename)

Cache folder: ./data/.cache already exists
File ./data/COCA_10140_021624.csv exists
COCA_10140_021624.csv


In [14]:
# Read in the Woodcock Johnson III Form A dataset for assessment
with open("data/wj_iii_form_a.json", "r") as f:
    wj3_json = json.load(f)

In [3]:
# Create a template dataframe to populate for each checkpoint
df = pd.DataFrame(
    columns=[
        "word_raw",
        "phon_target",
        "phon_prediction",
        "correct",
        "phon_target_features",
        "phon_prediction_features",
        "phon_prediction_probabilities",
        "global_encoding",
        "in_wj3",
        "in_traindata",
    ]
)

In [4]:
# read in the checkpoint file names. We will load one at a time
# and generate predictions of the wj3 assessments one at at ime
#root_2024-02-12_22h29m10508ms_chkpt004
#_chkpt001
checkpoints = glob.glob("models/root_2024-02-20_20h00m12987ms*.pth")
checkpoints.sort()
checkpoints=checkpoints[:-18]
checkpoints

['models/root_2024-02-20_20h00m12987ms_chkpt001.pth',
 'models/root_2024-02-20_20h00m12987ms_chkpt002.pth']

In [11]:
# Here we take all orthographic words from both WJ3 and the 
# original dataset and remove duplicates.

#df1=pd.read_csv( "/workspaces/ConnTextUL/data/random_kid_10140_021624.csv" )
df3=pd.read_csv("/workspaces/ConnTextUL/data/COCA_10140_021624.csv")
'''
df2=pd.read_csv("/workspaces/ConnTextUL/data/sidewalk_data.csv")
#
df4=pd.read_csv("/workspaces/ConnTextUL/data/kidwords_10140_021624.csv")
df5=pd.read_csv("/workspaces/ConnTextUL/data/random_adult_10140_021624.csv")
ccdf = pd.concat([df1, df2,df4,df5])
ccdf['word_raw']=ccdf['word_raw'].str.lower()
ccdf['word_raw']=ccdf['word_raw'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
ccdf=ccdf.dropna(subset=['word_raw'])
'''
#indices=ds.words
#split_index = int(0.8 * len(ds.words))
#train_indices = indices[:split_index]
#test_indices = indices[split_index:]
#all_words=wj3_json.keys()
#all_words = set(wj3_json.keys())
word_raw = 'word_raw'
#all_words = dfa[word_raw].tolist()
all_words=set(df3[word_raw])
#all_words.update(ds.words)  # Take the union of both sets
#in_train=[True if i in set(train_indices) else False for i in all_words ]
#in_test= [True if i in set(test_indices) else False for i in all_words] 
#data_f={"word_raw" : list(all_words) ,     "in_train":in_train, "in_test": in_test }
#df_f = pd.DataFrame(data_f)
#df_f.to_csv('model_02-12-2024_train_test.csv', index=False)
#all_words=set()
#all_words.update(df1[word_raw].tolist())
#all_words.update(df2[word_raw].tolist())
#all_words.update(df3[word_raw].tolist())
#all_words.update(df4[word_raw].tolist())
#all_words.update(ccdf[word_raw].tolist())

#df_f=pd.DataFrame(all_words)
#df_f.to_csv('all_words.csv', index=False)
#print(len(ccdf[word_raw].tolist()))
print(len(all_words))

2915


In [6]:
def timer_func(func): 
    # This function shows the execution time of  
    # the function object passed 
    def wrap_func(*args, **kwargs): 
        t1 = time() 
        result = func(*args, **kwargs) 
        t2 = time() 
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s') 
        return result 
    return wrap_func

In [9]:
#writer = pd.ExcelWriter("model_root_2024-02-12_22h29m10508ms_nonsense.xlsx", engine="openpyxl")
@timer_func
def running_comp(checkpoints,df):
    for checkpoint in tqdm.tqdm(checkpoints):
        csv_filename = f"test__{ds.data_filename}_model_{checkpoint[12:-15]}_checkpoint_{checkpoint[-7:-4]}.csv"
        with open(csv_filename, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            #writer.writerow(["word_raw", "phon_target", "phon_prediction", "correct", "phon_target_features", "phon_prediction_features", "phon_prediction_probabilities", "global_encoding"])
            
            # Write header row
            #writer.writerow(["word_raw", "phon_target", "phon_prediction", "correct", "phon_target_features", "phon_prediction_features", "phon_prediction_probabilities", "global_encoding"])
            # Empty the dataframe to prepare for the next checkpoint (aka excel page)
            df.drop(df.index, inplace=True)
            chkpt = pt.load(checkpoint)
            model = chkpt["model"]
            #In this loop, orth is the orthographic form of modelthe target word and
            #phon is the phonological form of the target word (not phonetic vectors)
            for orth in all_words:
                new_row = {}
                # Since all_words is a union of both WJ3 and the original dataset,
                # the phonological form of any orth in all_words is in either WJ3 or
                # the original cmudict dataset. We check both below
            # in_wj3 = orth in wj3_json.keys()
            # new_row["in_wj3"] = in_wj3
                in_traindata = orth in ds.words
                new_row["in_traindata"] = in_traindata
                #if in_traindata:
                phon = ds.cmudict[orth]
            # else:
                #    phon = wj3_json[orth]

                datum = ds.character_tokenizer.encode(orth)
                pred = chkpt["model"].generate(
                    "o2p",
                    datum["enc_input_ids"],
                    datum["enc_pad_mask"],
                    datum["enc_input_ids"],
                    datum["enc_pad_mask"],
                    deterministic=True,
                )
                # Save the original input orthography
                new_row["word_raw"] = orth
                # Save the target phonology for the above input orthography
                new_row["phon_target"] = ":".join(phon)
                # Remove the start and end tokens from each phonological vector
                # and convert them from tensors to lists
                phon_pred_features = [tensor.tolist() for tensor in pred["phon_tokens"][1:-1]]
                # Convert the phonological vectors to phonemes using Matt's handy dandy routine
                phon_pred = ds.phonology_tokenizer.traindata.convert_numeric_prediction(
                    phon_pred_features, phonology=True, hot_nodes=True
                )
                # Save the model's predicted pronunciation for this word. Phonemes are
                # separated by colons
                phon_pred = ["None" if p == None else p for p in phon_pred]
                new_row["phon_prediction"] = ":".join(phon_pred)
                # Save a boolean indicating whether the prediction was correct
                new_row["correct"] = new_row["phon_target"] == new_row["phon_prediction"]
                # Save the phonological features for the target phonology
                phon_target_features = ds.phonology_tokenizer.encode([orth])
                if phon_target_features:
                    phon_target_features = ";".join(
                        [
                            ":".join([str(v.item()) for v in vector])
                            for vector in phon_target_features["targets"][0]
                        ]
                    )
                else:
                    phon_target_features = "None"
                new_row["phon_target_features"] = phon_target_features
                # Save the phonological features for the predicted phonology
                phon_prediction_features = ";".join(
                    [
                        ":".join([str(int(v.item())) for v in vector])
                        for vector in pred["phon_vecs"][1:-1]
                    ]
                )
                new_row["phon_prediction_features"] = phon_prediction_features
                # Save the phonological probabilities for the predicted phonology
                phon_prediction_probabilities = ";".join(
                    [
                        ":".join([str(v.item()) for v in vector])
                        for vector in pred["phon_probs"][1:-1]
                    ]
                )
                new_row["phon_prediction_probabilities"] = phon_prediction_probabilities
                # Save the global encoding vector for the predicted phonology
                global_encoding = ":".join(
                    [str(v.item()) for v in pred["global_encoding"].squeeze()]
                )
                new_row["global_encoding"] = global_encoding
                new_row = pd.Series(new_row)
                new_row = new_row.to_frame().transpose()
                #writer.writerow([new_row["word_raw"], new_row["phon_target"], new_row["phon_prediction"], new_row["correct"], new_row["phon_target_features"], new_row["phon_prediction_features"], new_row["phon_prediction_probabilities"], new_row["global_encoding"]])
                df = pd.concat([df, new_row])
            df.to_csv(csvfile, index=False)
            #df.to_excel(writer, sheet_name="epoch " + checkpoint[-7:-4])
    #writer.book.save("model_root_2024-02-12_22h29m10508ms_nonsense.xlsx")

In [10]:
running_comp(checkpoints,df)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]


KeyError: 'suspects'